<a href="https://colab.research.google.com/github/adi0808/adi0808/blob/master/66degrees_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import json
from google.colab import userdata
from datetime import datetime
import sqlite3

kaggle_config = {
    "username": userdata.get("KAGGLE_USERNAME"),
    "key": userdata.get("KAGGLE_KEY")
}

os.makedirs("/root/.kaggle", exist_ok=True)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_config, f)

os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("kaggle.json created")

kaggle.json created


In [8]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
print("Authenticated successfully!")

Authenticated successfully!


In [9]:
api.dataset_download_files(
    "faresashraf1001/supermarket-sales",
    path="sample_data",
    unzip=True
)

print("Download complete!")

Dataset URL: https://www.kaggle.com/datasets/faresashraf1001/supermarket-sales
Download complete!


In [16]:
import pandas as pd

# reading csv data
df = pd.read_csv("sample_data/SuperMarket Analysis.csv")

# Renaming the columns to sinlk with the table columns
df = df.rename(columns={
    "Invoice ID": "invoice_id",
    "Branch": "branch",
    "City": "city",
    "Customer type": "customer_type",
    "Gender": "gender",
    "Product line": "product_line",
    "Unit price": "unit_price",
    "Quantity": "quantity",
    "Tax 5%": "tax",
    "Sales": "sales",
    "Date": "date",
    "Time": "time",
    "Payment": "payment",
    "cogs": "cogs",
    "gross margin percentage": "gross_margin_percentage",
    "gross income": "gross_income",
    "Rating": "rating"
})

# Casting to date type from object type
df['date'] = pd.to_datetime(df['date'])

# Creating 24 hours duration time from 12 hours format
df['time'] = pd.to_datetime(df['time'], format='%I:%M:%S %p').dt.time
df.head(2)

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax,sales,date,time,payment,cogs,gross_margin_percentage,gross_income,rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08:00,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29:00,Cash,76.40,4.761905,3.8200,9.6




> **Schema Design for 2 Dimension tables and 1 fact table**




**Dim_Customer**
- customer_id(primary key)
- customer_type
- gender
- branch
- city
- created_at

**Dim_Product**
- product_id(primary key)
- product_line
- unit_price
- created_at

**Fact_Sales**
- sales_id(primary key)
- invoice_id
- customer_id(foreign key)
- product_id(foreign key)
- quantity
- tax
- sales
- cogs
- gross_margin_percentage
- gross_income
- rating
- payment
- date
- time
- created_at



In [48]:
# SQL script for creating the tables.
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Customer Dimension
cursor.execute("""
CREATE TABLE Dim_Customer (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_type TEXT,
    gender TEXT,
    branch TEXT,
    city TEXT,
    created_at DATETIME
);
""")

# Product Dimension
cursor.execute("""
CREATE TABLE Dim_Product (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_line TEXT,
    unit_price REAL,
    created_at DATETIME
);
""")

# Fact Sales Table
cursor.execute("""
CREATE TABLE Fact_Sales (
    sales_id INTEGER PRIMARY KEY AUTOINCREMENT,
    invoice_id TEXT,
    customer_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    tax REAL,
    sales REAL,
    cogs REAL,
    gross_margin_percentage REAL,
    gross_income REAL,
    rating REAL,
    payment TEXT,
    date TEXT,
    time TEXT,
    created_at DATETIME,
    FOREIGN KEY (customer_id) REFERENCES Dim_Customer(customer_id),
    FOREIGN KEY (product_id) REFERENCES Dim_Product(product_id)
);
""")

conn.commit()


In [49]:
# Customer Dimension Transformation
dim_customer_df = df[['customer_type', 'gender', 'branch', 'city']].drop_duplicates()
dim_customer_df['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dim_customer_df = dim_customer_df.reset_index(drop=True)
print(dim_customer_df.head())
dim_customer_df.count()

  customer_type  gender branch       city           created_at
0        Member  Female   Alex     Yangon  2026-02-24 20:20:18
1        Normal  Female   Giza  Naypyitaw  2026-02-24 20:20:18
2        Normal  Female   Alex     Yangon  2026-02-24 20:20:18
3        Member  Female   Giza  Naypyitaw  2026-02-24 20:20:18
4        Member  Female  Cairo   Mandalay  2026-02-24 20:20:18


,0
customer_type,12
gender,12
branch,12
city,12
created_at,12


In [50]:
# Loading Customer data to table
conn = sqlite3.connect("sales.db")
dim_customer_df.to_sql(
    "Dim_Customer",
    conn,
    if_exists="append",
    index=False
)
conn.commit()
pd.read_sql("SELECT * FROM Dim_Customer limit 5;", conn)

,customer_id,customer_type,gender,branch,city,created_at
0,1,Member,Female,Alex,Yangon,2026-02-24 20:20:18
1,2,Normal,Female,Giza,Naypyitaw,2026-02-24 20:20:18
2,3,Normal,Female,Alex,Yangon,2026-02-24 20:20:18
3,4,Member,Female,Giza,Naypyitaw,2026-02-24 20:20:18
4,5,Member,Female,Cairo,Mandalay,2026-02-24 20:20:18


In [51]:
# Product Dimension Transformation
dim_product_df = df[['product_line', 'unit_price']].drop_duplicates()
from datetime import datetime

dim_product_df['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dim_product_df = dim_product_df.reset_index(drop=True)
print(dim_product_df.head())
dim_product_df.count()

             product_line  unit_price           created_at
0       Health and beauty       74.69  2026-02-24 20:20:27
1  Electronic accessories       15.28  2026-02-24 20:20:27
2      Home and lifestyle       46.33  2026-02-24 20:20:27
3       Health and beauty       58.22  2026-02-24 20:20:27
4       Sports and travel       86.31  2026-02-24 20:20:27


,0
product_line,993
unit_price,993
created_at,993


In [52]:
# Loading Product data to table
dim_product_df.to_sql(
    "Dim_Product",
    conn,
    if_exists="append",
    index=False
)
conn.commit()
pd.read_sql("SELECT * FROM Dim_Product limit 5;", conn)

,product_id,product_line,unit_price,created_at
0,1,Health and beauty,74.69,2026-02-24 20:20:27
1,2,Electronic accessories,15.28,2026-02-24 20:20:27
2,3,Home and lifestyle,46.33,2026-02-24 20:20:27
3,4,Health and beauty,58.22,2026-02-24 20:20:27
4,5,Sports and travel,86.31,2026-02-24 20:20:27


In [53]:
# Fact Sales data creation and transformation
dim_customer_db = pd.read_sql("SELECT customer_id, customer_type, gender, branch, city FROM Dim_Customer;", conn)
dim_product_db = pd.read_sql("SELECT product_id, product_line, unit_price FROM Dim_Product;", conn)
df = df.drop_duplicates()
fact_sales_df = df.merge(dim_customer_db,on=['customer_type', 'gender', 'branch', 'city'],
                how='left')
fact_sales_df = fact_sales_df.merge(dim_product_db,on=['product_line', 'unit_price'],
                how='left')
fact_sales_df['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
fact_sales_df = fact_sales_df[[
    'invoice_id',
    'customer_id',
    'product_id',
    'quantity',
    'tax',
    'sales',
    'cogs',
    'gross_margin_percentage',
    'gross_income',
    'rating',
    'payment',
    'date',
    'time',
    'created_at'
]]
fact_sales_df.head()
# fact_sales_df[['customer_id', 'product_id']].isnull().sum()

,invoice_id,customer_id,product_id,quantity,tax,sales,cogs,gross_margin_percentage,gross_income,rating,payment,date,time,created_at
0,750-67-8428,1,1,7,26.1415,548.9715,522.83,4.761905,26.1415,9.1,Ewallet,2019-01-05,13:08:00,2026-02-24 20:20:38
1,226-31-3081,2,2,5,3.8200,80.2200,76.40,4.761905,3.8200,9.6,Cash,2019-03-08,10:29:00,2026-02-24 20:20:38
2,631-41-3108,3,3,7,16.2155,340.5255,324.31,4.761905,16.2155,7.4,Credit card,2019-03-03,13:23:00,2026-02-24 20:20:38
3,123-19-1176,1,4,8,23.2880,489.0480,465.76,4.761905,23.2880,8.4,Ewallet,2019-01-27,20:33:00,2026-02-24 20:20:38
4,373-73-7910,1,5,7,30.2085,634.3785,604.17,4.761905,30.2085,5.3,Ewallet,2019-02-08,10:37:00,2026-02-24 20:20:38


In [54]:
# Loading Fact Sales data to table
fact_sales_df.to_sql(
    "Fact_Sales",
    conn,
    if_exists="append",
    index=False
)
conn.commit()
pd.read_sql("SELECT * FROM Fact_Sales limit 5;", conn)

,sales_id,invoice_id,customer_id,product_id,quantity,tax,sales,cogs,gross_margin_percentage,gross_income,rating,payment,date,time,created_at
0,1,750-67-8428,1,1,7,26.1415,548.9715,522.83,4.761905,26.1415,9.1,Ewallet,2019-01-05 00:00:00,13:08:00.000000,2026-02-24 20:20:38
1,2,226-31-3081,2,2,5,3.8200,80.2200,76.40,4.761905,3.8200,9.6,Cash,2019-03-08 00:00:00,10:29:00.000000,2026-02-24 20:20:38
2,3,631-41-3108,3,3,7,16.2155,340.5255,324.31,4.761905,16.2155,7.4,Credit card,2019-03-03 00:00:00,13:23:00.000000,2026-02-24 20:20:38
3,4,123-19-1176,1,4,8,23.2880,489.0480,465.76,4.761905,23.2880,8.4,Ewallet,2019-01-27 00:00:00,20:33:00.000000,2026-02-24 20:20:38
4,5,373-73-7910,1,5,7,30.2085,634.3785,604.17,4.761905,30.2085,5.3,Ewallet,2019-02-08 00:00:00,10:37:00.000000,2026-02-24 20:20:38


In [55]:
# Branch wise total sales, total gross income, and than rank branches based on total sales in descending order
pd.read_sql("""SELECT
    dc.branch,
    dc.city,
    round(SUM(fs.sales), 2) AS total_sales,
    round(SUM(fs.gross_income), 2) AS total_gross_income,
    COUNT(fs.invoice_id) AS total_transactions,
    RANK() OVER (ORDER BY SUM(fs.sales) DESC) AS sales_rank
FROM Fact_Sales fs
JOIN Dim_Customer dc
    ON fs.customer_id = dc.customer_id
JOIN Dim_Product dp
    ON fs.product_id = dp.product_id
GROUP BY dc.branch, dc.city
ORDER BY total_sales DESC;""", conn)

,branch,city,total_sales,total_gross_income,total_transactions,sales_rank
0,Giza,Naypyitaw,110568.71,5265.18,328,1
1,Alex,Yangon,106200.37,5057.16,340,2
2,Cairo,Mandalay,106197.67,5057.03,332,3


In [56]:
# Top 3 products for each branch based on total sales
pd.read_sql("""WITH product_sales AS (
    SELECT
        dc.branch,
        dp.product_line,
        round(SUM(fs.sales), 2) AS total_sales
    FROM Fact_Sales fs
    JOIN Dim_Customer dc
        ON fs.customer_id = dc.customer_id
    JOIN Dim_Product dp
        ON fs.product_id = dp.product_id
    GROUP BY dc.branch, dp.product_line
)

SELECT *
FROM (
    SELECT
        branch,
        product_line,
        total_sales,
        ROW_NUMBER() OVER (
            PARTITION BY branch
            ORDER BY total_sales DESC
        ) AS rn
    FROM product_sales
)
WHERE rn <= 3
ORDER BY branch, rn;""", conn)

,branch,product_line,total_sales,rn
0,Alex,Home and lifestyle,22417.20,1
1,Alex,Sports and travel,19372.70,2
2,Alex,Electronic accessories,18317.11,3
3,Cairo,Sports and travel,19988.20,1
4,Cairo,Health and beauty,19980.66,2
5,Cairo,Home and lifestyle,17549.16,3
6,Giza,Food and beverages,23766.85,1
7,Giza,Fashion accessories,21560.07,2
8,Giza,Electronic accessories,18968.97,3
